In [21]:
import os
import pandas as pd
import ftplib

In [25]:
#
# setup output directory
# 
output_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'

In [26]:
ftp = ftplib.FTP(host="download.brainimagelibrary.org")
ftp.login()

home_dir = ftp.pwd()
#remote_base = "biccn/zeng/luo/fMOST/cells"
remote_base = "56/77/567794f41ad2dccd"
ftp.cwd(remote_base)

dlist = []
ftp.dir( dlist.append )

dlist = [x.split() for x in dlist if x != '']
dlist = pd.DataFrame(dlist)
dlist = dlist[dlist[8].str.contains('mouseID')]

In [28]:
columns = ['Filename','Sample ID','AIBS LABTrack ID','HUST Sample ID','Reconstruction ID','Reconstruction Type','Cell Soma Z','Cell Soma X','Cell Soma Y']
df = pd.DataFrame([], columns = columns)
count = 0

for index,row in dlist.iterrows() :
    
    specimen = row[8]
    print(specimen)
    
    clist = specimen.split('-')
    hust_id = clist[1]
    labtrack_id = clist[0].split('_')[1]
    
    # print((labtrack_id,hust_id))
    
    # change into specimen directory
    ftp.cwd(specimen)
    #ftp.cwd('1')
    
    slist = []
    ftp.dir( slist.append )
    slist = [x.split() for x in slist if x != '']
    slist = pd.DataFrame(slist)
    slist = slist[slist[8].str.contains('.swc')]
    
    slist['Filename'] = slist[8]
    slist['Sample ID'] = specimen
    slist['AIBS LABTrack ID'] = labtrack_id
    slist['HUST Sample ID'] = hust_id
    
    slist['temp'] = [x.replace('.swc','') + '_' for x in slist[8]]
    slist['temp'] = [x.split('_') for x in slist['temp']]
    
    slist['Reconstruction ID'] = [x[0] for x in slist['temp']]
    slist['Reconstruction Type'] = None
    
    # finding the term "reg"
    #slist['Reconstruction Type'] = [x[3] for x in slist['temp']]
    slist.loc[slist['Filename'].str.contains('reg'),'Reconstruction Type'] = 'reg'
    
    slist['temp'] = [x[2].split('-') for x in slist['temp']]
    
    slist['Cell Soma Z'] = [int(x[0]) for x in slist['temp']]
    slist['Cell Soma X'] = [int(x[1].replace('X','')) for x in slist['temp']]
    slist['Cell Soma Y'] = [int(x[2].replace('Y','')) for x in slist['temp']]
    
    df = df.append(slist[columns], ignore_index = True)
    
    # reset to base directory
    ftp.cwd(home_dir)
    ftp.cwd(remote_base)
    
    count = count + 1
    

mouseID_210254-15257
mouseID_236174-16124
mouseID_297974-17109
mouseID_314107-17300
mouseID_321237-17302
mouseID_321244-17545
mouseID_325875-17543
mouseID_339951-17781
mouseID_339952-17782
mouseID_351327-17787
mouseID_351331-17788
mouseID_358361-18047
mouseID_373367-18454
mouseID_373368-18455
mouseID_373641-18462
mouseID_374706-18461
mouseID_374707-18452
mouseID_374712-18453
mouseID_378668-18470
mouseID_380470-191812
mouseID_381484-18457
mouseID_381487-18458
mouseID_381488-18464
mouseID_382148-18864
mouseID_383128-18465
mouseID_383680-18463
mouseID_394528-18867
mouseID_396476-18868
mouseID_396477-18869
mouseID_405426-182724
mouseID_405429-182725
mouseID_420489-191801
mouseID_423019-191803
mouseID_439168-191807


In [29]:
ftp.quit()

'221 Goodbye.'

In [30]:
df

,Filename,Sample ID,AIBS LABTrack ID,HUST Sample ID,Reconstruction ID,Reconstruction Type,Cell Soma Z,Cell Soma X,Cell Soma Y
0,1059283677_15257_2226-X16029-Y23953.swc,mouseID_210254-15257,210254,15257,1059283677,None,2226,16029,23953
1,1059284108_15257_4500-X16614-Y17147.swc,mouseID_210254-15257,210254,15257,1059284108,None,4500,16614,17147
2,1059284262_15257_4149-X16164-Y21238.swc,mouseID_210254-15257,210254,15257,1059284262,None,4149,16164,21238
3,1059286053_15257_4021-X16401-Y21556.swc,mouseID_210254-15257,210254,15257,1059286053,None,4021,16401,21556
4,1059287090_15257_4397-X16794-Y21812.swc,mouseID_210254-15257,210254,15257,1059287090,None,4397,16794,21812
...,...,...,...,...,...,...,...,...,...
3477,1119569830_191807_5178-X5002-Y23078__reg.swc,mouseID_439168-191807,439168,191807,1119569830,reg,5178,5002,23078
3478,1119569832_191807_5199-X3288-Y11643__reg.swc,mouseID_439168-191807,439168,191807,1119569832,reg,5199,3288,11643
3479,1119569834_191807_5629-X4661-Y9410__reg.swc,mouseID_439168-191807,439168,191807,1119569834,reg,5629,4661,9410
3480,1119569836_191807_5811-X3931-Y20674__reg.swc,mouseID_439168-191807,439168,191807,1119569836,reg,5811,3931,20674


In [31]:
#
# write out data frame
#
output_path = os.path.join(output_directory,'reconstruction_inventory.csv')
df.to_csv( output_path, index=False )